# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

### 1. imported this to create functions for spark sql

In [11]:
from pyspark.sql import functions as F

### 2. Building the “same-state” counts CTE in PySpark

- Joining  twice: c (citations) → p_cit (attributes for the citing patent) on CITING = PATENT & c → p_ced (attributes for the cited patent) on CITED = PATENT
Using aliases (alias) keeps the two patent joins clear. 
- Filtering with:U.S. patents only (COUNTRY = 'US'), Both states present/non-empty (isNotNull() and trim(...) != ''). 
- Same state match between citer and cited (p_cit.POSTATE == p_ced.POSTATE).
- Exclude self-citations (CITING != CITED).
- Group & aggregate: count rows per CITING to get SAME_STATE.

In [9]:
same_state_counts = (
    citations.alias('c')
    .join(patents.alias('p_cit'), F.col('c.CITING') == F.col('p_cit.PATENT'))
    .join(patents.alias('p_ced'), F.col('c.CITED') == F.col('p_ced.PATENT'))
    .filter(
        (F.col('p_cit.COUNTRY') == 'US') &
        (F.col('p_ced.COUNTRY') == 'US') &
        (F.trim(F.col('p_cit.POSTATE')) != '') & F.col('p_cit.POSTATE').isNotNull() &
        (F.trim(F.col('p_ced.POSTATE')) != '') & F.col('p_ced.POSTATE').isNotNull() &
        (F.col('p_cit.POSTATE') == F.col('p_ced.POSTATE')) &
        (F.col('c.CITING') != F.col('c.CITED'))
    )
    .groupBy('CITING')
    .agg(F.count('*').alias('SAME_STATE'))
)

### 3. Final joining of SAME_STATE COLUMN

- Joining counts back to patents and picking the top 10
- Joining so each citing patent row carries its SAME_STATE count.
- Filter to U.S. patents with a valid state.
- Sort by highest SAME_STATE, tie-break by patent id, then limit to 10 rows.
- Selecting all patent columns plus the SAME_STATE metric. 

In [10]:
final_result = (
    patents.alias('p')
    .join(same_state_counts, F.col('p.PATENT') == F.col('CITING'))
    .filter(
        (F.col('p.COUNTRY') == 'US') &
        (F.trim(F.col('p.POSTATE')) != '') & F.col('p.POSTATE').isNotNull()
    )
    .orderBy(F.col('SAME_STATE').desc(), F.col('p.PATENT'))
    .limit(10)
    .select(F.col('p.*'), F.col('SAME_STATE'))
)

final_result.show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  NULL|   326|  4|    46|  159|       0|     1.0|   NULL|  0.6186|    NULL|  4.8868|  0.0455|   0.044|    NULL|    NULL|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  NULL|   114|  5|    55|  200|       0|   0.995|   NULL|  0.7201|    NULL|   12.45|     0.0|     0.0|    NULL|    NULL|       103|
|6008204| 1999|14606|   1998| 